In [1]:
import csv
import pandas as pd
import numpy as np
import torch
from datetime import datetime

#Import Huggingface models
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Plotly Libraries
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [26]:
body='The new fresh face of the housing development industry, NMC Real Estate announces that it has begun operations for construction of its 600 houses.NMC which was founded by Meseret Mekonnen stated that is project located in front of Tulu Dimtu, Aqaqi-Qaliti Sub City will have a total area of 35,000 square meters.During a press conference and official signing ceremony held on Saturday June 24, the real estate company announced that the construction of the 600 houses had already commenced with about 20 villa units well in pace to be finalized soon.As per the plan, the project will include 30 G plus 2 villas and 14 apartments with 12 stories.“In total the project will consume 8.5 billion birr,” said representatives of the company at the press conference.The contractor leading the project is non-other than Huahong Construction, a Chinese company that is not new to similar projects in Ethiopia. Currently, about 400 employees including expats are involved on the construction.Internal road connection as well as the drilling of water well has already been accomplished. According to the company, the project is scheduled to be accomplished within two years’ time.“Regarding addressing the housing scheme the company would have similar projects in other cities in the country,” the company leaders explained.NMC Real Estate stated that it decided to join the real state sector to play its part on the housing problem that is one of the major socioeconomic challenges in the country, mainly in major cities.The investor is involved on different businesses including the hospitality industry, while its ventures on the real estate industry are relatively new.In the ten year plan, the government has projected about 4.4 million houses will be built .Of that, the major stake of about 80 percent will be handled by the private sector.The real estate industry was one of the mistrusted sectors for buyers up until recently when new comers changed the trend by keeping their words on transferring houses on time.'

In [27]:
classifier = pipeline('sentiment-analysis', model='ProsusAI/finbert')
classifier(body)

[{'label': 'neutral', 'score': 0.893200159072876}]

In [45]:
df = pd.read_csv('localnews.csv')
df_body = df['Body']

df_array = np.array(df_body)
df_bodylist = list(df_array)

In [46]:
# Processing this cell takes about 30 mins, result however is saved in finbert_result.csv


df_finbertResult = pd.DataFrame(columns=['Index', 'fpositive', 'fnegative', 'fneutral', 'score', 'label'])

for i, body in enumerate(df_bodylist):
    inputs = tokenizer(body, padding = True, truncation = True, return_tensors='pt') #tokenize text to be sent to model
    outputs = model(**inputs)

    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    print("Analysis " + str(i) +" done.")
    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()
    comparedict = {'positive':positive, 'negative':negative, 'neutral':neutral}
    score = max(positive, negative, neutral)
    label = max(comparedict, key = lambda k: comparedict[k])
    result = {"Index":i, "fpositive":positive, "fnegative":negative, "fneutral":neutral, "score":score, "label":label}
    df_toappend = pd.DataFrame(data=result, index=[i])
    df_finbertResult = pd.concat([df_finbertResult, df_toappend])

Analysis 0 done.
Analysis 1 done.
Analysis 2 done.
Analysis 3 done.
Analysis 4 done.
Analysis 5 done.
Analysis 6 done.
Analysis 7 done.
Analysis 8 done.
Analysis 9 done.
Analysis 10 done.
Analysis 11 done.
Analysis 12 done.
Analysis 13 done.
Analysis 14 done.
Analysis 15 done.
Analysis 16 done.
Analysis 17 done.
Analysis 18 done.
Analysis 19 done.
Analysis 20 done.
Analysis 21 done.
Analysis 22 done.
Analysis 23 done.
Analysis 24 done.
Analysis 25 done.
Analysis 26 done.
Analysis 27 done.
Analysis 28 done.
Analysis 29 done.
Analysis 30 done.
Analysis 31 done.
Analysis 32 done.
Analysis 33 done.
Analysis 34 done.
Analysis 35 done.
Analysis 36 done.
Analysis 37 done.
Analysis 38 done.
Analysis 39 done.
Analysis 40 done.
Analysis 41 done.
Analysis 42 done.
Analysis 43 done.
Analysis 44 done.
Analysis 45 done.
Analysis 46 done.
Analysis 47 done.
Analysis 48 done.
Analysis 49 done.
Analysis 50 done.
Analysis 51 done.
Analysis 52 done.
Analysis 53 done.
Analysis 54 done.
Analysis 55 done.
An

In [47]:
df_finbertResult.head(20)

,Index,fpositive,fnegative,fneutral,score,label
0,0,0.753152,0.008772,0.238076,0.753152,positive
1,1,0.076639,0.030161,0.893200,0.893200,neutral
2,2,0.603660,0.009562,0.386778,0.603660,positive
3,3,0.656810,0.237762,0.105428,0.656810,positive
4,4,0.887694,0.007952,0.104354,0.887694,positive
5,5,0.282715,0.008630,0.708655,0.708655,neutral
6,6,0.814691,0.008282,0.177026,0.814691,positive
7,7,0.018252,0.852250,0.129498,0.852250,negative
8,8,0.777893,0.009262,0.212844,0.777893,positive
9,9,0.699103,0.008803,0.292094,0.699103,positive


In [3]:
df_finbertResult['label'].value_counts()

NameError: name 'df_finbertResult' is not defined

In [49]:
df_finbertResult.to_csv('finbert_result.csv')

In [4]:
df_finbertResult = pd.read_csv('finbert_result.csv')
fig = px.pie(df_finbertResult, names='label', 
             height=300, width=600, 
             hole=0.7, 
             title='Sentiment Overview, finBERT',
             color_discrete_sequence=['#CAE7B9', '#EB9486', '#F3DE8A'])
fig.update_layout(plot_bgcolor='#fafafa', paper_bgcolor='#fafafa')
fig.show()